In [262]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph,END
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

load_dotenv()

True

In [263]:
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [264]:
# model = ChatGroq(temperature=0, model="llama3-70b-8192", streaming=False)
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
# model = ChatAnthropic(model="claude-3-haiku-20240307")

In [265]:
model.invoke('hola')

AIMessage(content='¡Hola! ¿En qué puedo ayudarte hoy?', response_metadata={'finish_reason': 'stop'}, id='run-14309d53-6d45-405d-85bc-2fc23aad3d60-0')

In [266]:
@tool
def multiply(first_number: int, second_number: int):
    """Multiplica dos números."""
    return first_number * second_number



In [267]:
model_with_tools = model.bind(tools=[convert_to_openai_tool(multiply)])
response = model_with_tools.invoke('Cuanto es 35 * 46?')
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_A1Ld6uHuMaf51YcnjCTtFn3z', 'function': {'arguments': '{"first_number":35,"second_number":46}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-292af45b-2dc9-4fb1-9ec8-05fdc0ef6b58-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 35, 'second_number': 46}, 'id': 'call_A1Ld6uHuMaf51YcnjCTtFn3z'}])

In [268]:
tools = [multiply]
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke('Cuanto es 35 * 46?')
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_keiMXTOXLkdf73YBorW7XngL', 'function': {'arguments': '{"first_number":35,"second_number":46}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-eaf023a9-1949-45a0-924a-fe83d75b103e-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 35, 'second_number': 46}, 'id': 'call_keiMXTOXLkdf73YBorW7XngL'}])

In [269]:
# tools = [multiply]
# model_with_tools = model.bind_tools(tools)
# response = model_with_tools.invoke('Que es un LLM?')
# response

In [270]:
tool_calls = response.additional_kwargs.get('tool_calls')
tool_calls

[{'index': 0,
  'id': 'call_keiMXTOXLkdf73YBorW7XngL',
  'function': {'arguments': '{"first_number":35,"second_number":46}',
   'name': 'multiply'},
  'type': 'function'}]

In [271]:
tool_calls = response.additional_kwargs.get('tool_calls')
tool_calls

[{'index': 0,
  'id': 'call_keiMXTOXLkdf73YBorW7XngL',
  'function': {'arguments': '{"first_number":35,"second_number":46}',
   'name': 'multiply'},
  'type': 'function'}]

In [272]:
type(tool_calls)

list

In [273]:
for tool_call in tool_calls:
    print('Function Name:',tool_call.get('function').get('name'))
    print('Function Arguments:',tool_call.get('function').get('arguments'))
    print(tool_call)

Function Name: multiply
Function Arguments: {"first_number":35,"second_number":46}
{'index': 0, 'id': 'call_keiMXTOXLkdf73YBorW7XngL', 'function': {'arguments': '{"first_number":35,"second_number":46}', 'name': 'multiply'}, 'type': 'function'}


### Construcción del Gráfico con llamada condicional a aristas y herramientas

In [274]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [275]:
def invoke_model(state):
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    return {"messages":[model_with_tools.invoke(question)]}

In [276]:
def invoke_tool(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    for tool_call in tool_calls:
        if tool_call.get("function").get("name") == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No se ha encontrado ninguna entrada para sumar.")

    res = multiply.invoke(
        json.loads(multiply_call.get("function").get("arguments"))
        # Convierte una cadena JSON a un diccionario de Python '{"first_number": 35, "second_number": 46}'
    )

    return {"messages" : [res]
    }

In [277]:
def router(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "end"

In [278]:
graph = StateGraph(AgentState)
graph.add_node("agent", invoke_model)
graph.add_node("tool", invoke_tool)
graph.add_edge("tool", END)
graph.add_conditional_edges("agent", router, {
    "multiply": "tool",
    "end": END,
})
graph.set_entry_point("agent")
app = graph.compile()

In [279]:
output = app.invoke({"messages": ["Cuanto es 123 * 456?"]})

In [280]:
output

{'messages': ['Cuanto es 123 * 456?',
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_YQPSJZYOQy401D0I8x9GjFMx', 'function': {'arguments': '{"first_number":123,"second_number":456}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-f67db5a5-21dc-4e9f-9f58-1981cfd2f255-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_YQPSJZYOQy401D0I8x9GjFMx'}]),
  56088]}

In [281]:
output['messages'][-1]

56088

In [282]:
output = app.invoke({"messages": ["Que es LLM?"]})

In [283]:
output

{'messages': ['Que es LLM?',
  AIMessage(content='Lo siento, pero no tengo información sobre "LLM". ¿Hay algo más en lo que pueda ayudarte?', response_metadata={'finish_reason': 'stop'}, id='run-599797b8-33ad-41a1-84bf-3b7c87676a2d-0')]}

In [284]:
print(output['messages'][-1])

content='Lo siento, pero no tengo información sobre "LLM". ¿Hay algo más en lo que pueda ayudarte?' response_metadata={'finish_reason': 'stop'} id='run-599797b8-33ad-41a1-84bf-3b7c87676a2d-0'
